In [13]:
import pandas as pd
import re
from collections import defaultdict, Counter
import difflib


class IntelligentProductGrouper:
    def __init__(self):
        # Padrões para remover (especificações técnicas que devem ser generalizadas)
        self.remove_patterns = [
            # Dosagens e concentrações
            r"\b\d+\s*(mg|mcg|g|ml|l|%)\b",
            r"\b\d+/\d+\s*(mg|mcg|g|ml|l)\b",
            # Quantidades de comprimidos/cápsulas
            r"\b\d+\s*(comp|compr|comprimidos|caps|cápsulas|sachês|saches)\b",
            r"\b\d+\s*(un|unid|unidades|pç|peças|und)\b",
            # Modelos e códigos específicos
            r"\b[A-Z]+\d+[A-Z]*\b",  # Ex: NX6325, RG140, etc.
            r"\b\d+[A-Z]+\d*\b",  # Ex: 170H, 4MG, etc.
            # Especificações técnicas de eletrônicos
            r"\b\d+\s*(gb|mb|kb|ghz|mhz|rpm|dpi)\b",
            r'\b\d+\s*(pol|polegadas|")\b',
            r"\bcom\s+pen\s+drive\b",
            r"\busb\s+\d+.*?\b",
            # Códigos de barras e referências
            r"\b\d{4,}\b",  # Números longos (4+ dígitos)
            # Especificações de tempo/voltagem
            r"\b\d+\s*(tempos|v|volts|w|watts|hp|cv)\b",
            r"\b\d+\.\d+\s*(kva|l|v)\b",
            # Especificações médicas específicas
            r"\bc/\d+\b",  # Ex: C/30, C/60
            r"\bha\s+c/\s*\d+\b",
        ]

        # Padrões para preservar (diferenciações importantes)
        self.preserve_patterns = [
            # Cores
            r"\b(azul|vermelho|verde|amarelo|preto|branco|rosa|roxo|laranja|marrom|cinza|dourado|prateado)\b",
            r"\b(blue|red|green|yellow|black|white|pink|purple|orange|brown|gray|gold|silver)\b",
            # Tipos de embalagem importantes
            r"\b(resma|pacote|caixa|frasco|tubo|bisnaga|spray)\b",
            # Tipos de papel específicos
            r"\b(a4|a3|a5|ofício|carta)\b",
            # Variações importantes de produto
            r"\b(original|premium|light|diet|zero|plus|pro|max)\b",
            r"\b(com|sem)\s+\w+",
            # Especificações de tamanho quando relevantes
            r"\b(pequeno|médio|grande|p|m|g|pp|gg)\b",
            r"\b(small|medium|large|xl|xxl|xs)\b",
        ]

    def normalize_product_name(self, text):
        """
        Normaliza o nome do produto removendo especificações técnicas
        mas preservando diferenciações importantes
        """
        if pd.isna(text) or not text.strip():
            return ""

        text = str(text).upper().strip()
        original_text = text

        # Extrair características a preservar ANTES de remover especificações
        preserved_features = []
        for pattern in self.preserve_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            preserved_features.extend(matches)

        # Remover especificações técnicas
        for pattern in self.remove_patterns:
            text = re.sub(pattern, "", text, flags=re.IGNORECASE)

        # Limpeza geral
        text = re.sub(r"\s+", " ", text)  # Múltiplos espaços
        text = re.sub(r"[^\w\s]", " ", text)  # Caracteres especiais
        text = re.sub(r"\b\w{1,2}\b", "", text)  # Palavras muito pequenas

        # Remover palavras comuns não essenciais
        stop_words = [
            "DE",
            "DA",
            "DO",
            "COM",
            "SEM",
            "PARA",
            "EM",
            "E",
            "OU",
            "A",
            "O",
            "AS",
            "OS",
        ]
        words = [
            word for word in text.split() if word not in stop_words and len(word) > 2
        ]

        # Reconstruir nome base
        base_name = " ".join(words).strip()

        # Adicionar características preservadas importantes
        important_features = []
        for feature in preserved_features:
            feature_clean = feature.lower().strip()
            if feature_clean and len(feature_clean) > 2:
                important_features.append(feature_clean)

        if important_features:
            # Remover duplicatas mantendo ordem
            unique_features = []
            for feat in important_features:
                if feat not in unique_features:
                    unique_features.append(feat)
            base_name += " " + " ".join(unique_features)

        return base_name.strip()

    def group_similar_products(self, df, product_col, similarity_threshold=0.85):
        """
        Agrupa produtos similares baseado em nome normalizado e similaridade
        """
        print(f"🤖 Iniciando agrupamento inteligente de produtos...")
        print(f"📊 Total de produtos: {len(df)}")

        # Normalizar nomes dos produtos
        df_work = df.copy()
        df_work["normalized_name"] = df_work[product_col].apply(
            self.normalize_product_name
        )

        # Remover produtos com nomes vazios após normalização
        df_work = df_work[df_work["normalized_name"].str.strip() != ""].copy()
        print(f"📊 Produtos com nomes válidos: {len(df_work)}")

        # Agrupar por nome normalizado exato
        exact_groups = df_work.groupby("normalized_name")

        grouped_products = []
        similarity_groups = []

        print(f"🔍 Processando grupos de produtos similares...")

        for name, group in exact_groups:
            if len(group) > 1:
                # Múltiplos produtos com o mesmo nome normalizado
                # Escolher o representante mais completo
                representative = self.choose_best_representative(group, product_col)
                grouped_products.append(
                    {
                        "produto_final": representative[product_col],
                        "normalized_name": name,
                        "produtos_agrupados": list(group[product_col].values),
                        "quantidade_agrupada": len(group),
                        "tipo_agrupamento": "exato",
                    }
                )
            else:
                # Produto único, manter como está
                representative = group.iloc[0]
                grouped_products.append(
                    {
                        "produto_final": representative[product_col],
                        "normalized_name": name,
                        "produtos_agrupados": [representative[product_col]],
                        "quantidade_agrupada": 1,
                        "tipo_agrupamento": "único",
                    }
                )

        # Segunda passagem: agrupar produtos com alta similaridade
        unique_names = [
            g["normalized_name"]
            for g in grouped_products
            if g["quantidade_agrupada"] == 1
        ]

        print(
            f"🔍 Analisando similaridade entre {len(unique_names)} produtos únicos..."
        )

        processed_indices = set()
        for i, name1 in enumerate(unique_names):
            if i in processed_indices:
                continue

            similar_group = [i]
            for j, name2 in enumerate(unique_names[i + 1 :], i + 1):
                if j in processed_indices:
                    continue

                similarity = difflib.SequenceMatcher(None, name1, name2).ratio()
                if similarity >= similarity_threshold:
                    similar_group.append(j)
                    processed_indices.add(j)

            if len(similar_group) > 1:
                # Agrupar produtos similares
                similar_products = [grouped_products[idx] for idx in similar_group]
                all_original_products = []
                for sp in similar_products:
                    all_original_products.extend(sp["produtos_agrupados"])

                # Escolher melhor representante
                temp_df = df_work[df_work[product_col].isin(all_original_products)]
                representative = self.choose_best_representative(temp_df, product_col)

                # Criar novo grupo consolidado
                similarity_groups.append(
                    {
                        "produto_final": representative[product_col],
                        "normalized_name": representative["normalized_name"],
                        "produtos_agrupados": all_original_products,
                        "quantidade_agrupada": len(all_original_products),
                        "tipo_agrupamento": "similaridade",
                    }
                )

                # Marcar como processados
                for idx in similar_group:
                    processed_indices.add(idx)
            else:
                processed_indices.add(i)

        # Combinar resultados
        final_groups = []

        # Adicionar grupos exatos com múltiplos produtos
        for group in grouped_products:
            if group["quantidade_agrupada"] > 1:
                final_groups.append(group)

        # Adicionar grupos por similaridade
        final_groups.extend(similarity_groups)

        # Adicionar produtos únicos não agrupados
        for i, group in enumerate(grouped_products):
            if group["quantidade_agrupada"] == 1 and i not in processed_indices:
                final_groups.append(group)

        return final_groups

    def choose_best_representative(self, group, product_col):
        """
        Escolhe o melhor representante de um grupo de produtos similares
        """
        # Critérios de qualidade:
        # 1. Descrição mais completa (mais palavras úteis)
        # 2. Menos números/códigos específicos
        # 3. Mais comum (se houver repetições)

        group = group.copy()

        # Calcular score de qualidade para cada produto
        def quality_score(text):
            if pd.isna(text):
                return 0

            text = str(text)
            score = 0

            # Mais palavras = melhor
            words = [w for w in text.split() if len(w) > 2]
            score += len(words) * 2

            # Menos números específicos = melhor
            numbers = len(re.findall(r"\d+", text))
            score -= numbers * 0.5

            # Palavras em maiúsculo (nomes de marca) = melhor
            caps_words = len(re.findall(r"\b[A-Z]{2,}\b", text))
            score += caps_words * 1.5

            return score

        group["quality_score"] = group[product_col].apply(quality_score)

        # Ordenar por qualidade e pegar o melhor
        best = group.sort_values(["quality_score"], ascending=False).iloc[0]
        return best

    def create_final_dataframe(self, original_df, groups, product_col):
        """
        Cria o DataFrame final com produtos agrupados
        """
        print(f"📋 Criando DataFrame final...")

        # Criar lista de produtos finais
        final_products = []
        grouping_info = []

        for group in groups:
            # Encontrar linha original do produto representativo
            original_row = (
                original_df[original_df[product_col] == group["produto_final"]]
                .iloc[0]
                .copy()
            )
            final_products.append(original_row)

            # Informações do agrupamento
            grouping_info.append(
                {
                    "produto_final": group["produto_final"],
                    "produtos_originais": "; ".join(group["produtos_agrupados"]),
                    "quantidade_agrupada": group["quantidade_agrupada"],
                    "tipo_agrupamento": group["tipo_agrupamento"],
                }
            )

        # Criar DataFrame final
        df_final = pd.DataFrame(final_products).reset_index(drop=True)
        df_grouping = pd.DataFrame(grouping_info)

        return df_final, df_grouping

    def analyze_grouping_results(self, groups, original_count):
        """
        Analisa e exibe estatísticas do agrupamento
        """
        print(f"\n📊 RESULTADOS DO AGRUPAMENTO:")
        print(f"=" * 50)

        total_grouped = sum(g["quantidade_agrupada"] for g in groups)
        final_count = len(groups)
        reduction = original_count - final_count

        print(f"📈 Produtos originais: {original_count}")
        print(f"📈 Produtos finais: {final_count}")
        print(f"🗑️  Produtos agrupados: {reduction}")
        print(f"📊 Redução: {(reduction/original_count)*100:.1f}%")

        # Estatísticas por tipo de agrupamento
        type_stats = defaultdict(int)
        big_groups = []

        for group in groups:
            type_stats[group["tipo_agrupamento"]] += 1
            if group["quantidade_agrupada"] > 5:
                big_groups.append(group)

        print(f"\n🏷️  TIPOS DE AGRUPAMENTO:")
        for tipo, count in type_stats.items():
            print(f"  {tipo}: {count} grupos")

        # Mostrar maiores agrupamentos
        if big_groups:
            print(f"\n🔝 MAIORES AGRUPAMENTOS:")
            big_groups.sort(key=lambda x: x["quantidade_agrupada"], reverse=True)
            for group in big_groups[:10]:
                print(
                    f"  {group['produto_final']} ({group['quantidade_agrupada']} produtos)"
                )

        return {
            "original_count": original_count,
            "final_count": final_count,
            "reduction": reduction,
            "reduction_percent": (reduction / original_count) * 100,
        }


def main():
    # Exemplo de uso
    PATH = "tabelaInicial.xlsx"

    # Carregar dados
    if PATH.endswith(".csv"):
        df = pd.read_csv(PATH, dtype=str, encoding="utf-8")
    else:
        df = pd.read_excel(PATH, dtype=str)

    print(f"📁 Arquivo carregado: {PATH}")
    print(f"📊 Total de linhas: {len(df)}")
    print(f"🗂️  Colunas: {list(df.columns)}")

    # Usar apenas uma amostra para teste (remova .head() para processar tudo)
    df_sample = df.head(1000)

    # Inicializar agrupador
    grouper = IntelligentProductGrouper()

    # Realizar agrupamento
    groups = grouper.group_similar_products(
        df_sample, "Produto/Serviço", similarity_threshold=0.8
    )

    # Criar DataFrame final
    df_final, df_grouping = grouper.create_final_dataframe(
        df_sample, groups, "Produto/Serviço"
    )

    # Analisar resultados
    stats = grouper.analyze_grouping_results(groups, len(df_sample))

    # Salvar resultados
    df_final.to_csv("produtos_agrupados.csv", index=False, encoding="utf-8")
    df_grouping.to_csv("detalhes_agrupamento.csv", index=False, encoding="utf-8")

    print(f"\n💾 Arquivos salvos:")
    print(f"  - produtos_agrupados.csv (produtos finais)")
    print(f"  - detalhes_agrupamento.csv (detalhes do agrupamento)")

    return df_final, df_grouping, stats


if __name__ == "__main__":
    df_final, df_grouping, stats = main()

📁 Arquivo carregado: tabelaInicial.xlsx
📊 Total de linhas: 1000
🗂️  Colunas: ['Código', 'Grupo', 'Produto/Serviço', 'Descrição', 'UND']
🤖 Iniciando agrupamento inteligente de produtos...
📊 Total de produtos: 1000
📊 Produtos com nomes válidos: 993
🔍 Processando grupos de produtos similares...
🔍 Analisando similaridade entre 597 produtos únicos...
📋 Criando DataFrame final...

📊 RESULTADOS DO AGRUPAMENTO:
📈 Produtos originais: 1000
📈 Produtos finais: 332
🗑️  Produtos agrupados: 668
📊 Redução: 66.8%

🏷️  TIPOS DE AGRUPAMENTO:
  exato: 139 grupos
  similaridade: 83 grupos
  único: 110 grupos

🔝 MAIORES AGRUPAMENTOS:
  ÔNIBUS TRANSPORTE ESCOLAR - R02EJPF (49 produtos)
  METFORMINA XR 500C/60 CP (8 produtos)
  METOPROLOL SUCC 25 MG C/30 CPR (7 produtos)
  NISTANINA 25.000 UI/G CREME VAGINAL (7 produtos)
  METILDOPA 250 MG. (6 produtos)
  METOPROLOL SUCC 25 MG C/30 CPR (6 produtos)
  OMEPRAZOL 10 MG C/ 14 COMP (6 produtos)
  MESA PLASTICA REDONDA BRANCA/LEBLON/ (6 produtos)
  NEOSTRATA MINESO